**Frozen Lake**

In this notebook, we implement Q-learning on OpenAI Gym to the Frozen Lake environment (Lapan book, chapter 6)

In [1]:
#!/usr/bin/env python3
import gym
import collections
!pip install tensorboardX
from tensorboardX import SummaryWriter

ENV_NAME = "FrozenLake-v1"
GAMMA = 0.9
ALPHA = 0.2
TEST_EPISODES = 20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.6 MB/s eta 0:00:00


Define Agent Class

In [2]:
class Agent:
    def __init__(self):
      # Intitialize Agent
      self.env = gym.make(ENV_NAME)
      self.state = self.env.reset()
      self.values = collections.defaultdict(float)

    def sample_env(self):
      # We sample a random action from the action space and return the tupple
      # [old_state, action, reward, new_state]
      action = self.env.action_space.sample()
      old_state = self.state
      new_state, reward, is_done, _ = self.env.step(action)
      self.state = self.env.reset() if is_done else new_state
      return old_state, action, reward, new_state

    def best_value_and_action(self, state):
      # Given a state, we search for the best action (having the best value)
      best_value, best_action = None, None
      for action in range(self.env.action_space.n):
          action_value = self.values[(state, action)]
          if best_value is None or best_value < action_value:
              best_value = action_value
              best_action = action
      return best_value, best_action

    def value_update(self, s, a, r, next_s):
      # Q-Learning update rule
      best_v, _ = self.best_value_and_action(next_s)
      new_v = r + GAMMA * best_v
      old_v = self.values[(s, a)]
      self.values[(s, a)] = old_v * (1-ALPHA) + new_v * ALPHA

    def play_episode(self, env):
      # Run one episode to test a policy
      total_reward = 0.0
      state = env.reset()
      while True:
          _, action = self.best_value_and_action(state)
          new_state, reward, is_done, _= env.step(action)
          total_reward += reward
          if is_done:
              break
          state = new_state
      return total_reward

Run Q-Learning

In [4]:
test_env = gym.make(ENV_NAME)
agent = Agent()
writer = SummaryWriter(comment="-q-learning")

iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    s, a, r, next_s = agent.sample_env()
    agent.value_update(s, a, r, next_s)

    reward = 0.0
    for _ in range(TEST_EPISODES):
        reward += agent.play_episode(test_env)
    reward /= TEST_EPISODES
    writer.add_scalar("reward", reward, iter_no)
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (
            best_reward, reward))
        best_reward = reward
    if reward > 0.95:
        print("Solved in %d iterations!" % iter_no)
        break
writer.close()


Best reward updated 0.000 -> 0.050
Best reward updated 0.050 -> 0.100
Best reward updated 0.100 -> 0.150
Best reward updated 0.150 -> 0.200
Best reward updated 0.200 -> 0.300
Best reward updated 0.300 -> 0.350
Best reward updated 0.350 -> 0.450
Best reward updated 0.450 -> 0.500
Best reward updated 0.500 -> 0.600
Best reward updated 0.600 -> 0.650
Best reward updated 0.650 -> 0.750
Best reward updated 0.750 -> 0.800
Best reward updated 0.800 -> 0.850
Best reward updated 0.850 -> 0.900
Best reward updated 0.900 -> 0.950
Best reward updated 0.950 -> 1.000
Solved in 19176 iterations!
